In [1]:
!pip install pyspark
!pip install pyarrow
!pip install -q findspark

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("logistic").getOrCreate()

In [3]:
from pyspark.ml.classification import LogisticRegression

In [4]:
df=spark.read.format("libsvm").load("/content/sample_libsvm_data.txt")

In [5]:
df.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [6]:
logistic_model=LogisticRegression()

In [7]:
train_model=logistic_model.fit(df)

In [8]:
summary=train_model.summary

In [9]:
summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [10]:
summary.predictions.select("label","prediction").show()

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
+-----+----------+
only showing top 20 rows



In [11]:
train_data,test_data=df.randomSplit([0.7,0.3])

In [12]:
final_model=LogisticRegression()

In [13]:
final_fit=final_model.fit(train_data)

In [14]:
predictions_and_labels=final_fit.evaluate(test_data)

In [15]:
predictions_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[21.6249999590862...|[0.99999999959413...|       0.0|
|  0.0|(692,[100,101,102...|[11.3447938486260...|[0.99998816921682...|       0.0|
|  0.0|(692,[123,124,125...|[31.0397014336930...|[0.99999999999996...|       0.0|
|  0.0|(692,[126,127,128...|[15.8566837667438...|[0.99999987012376...|       0.0|
|  0.0|(692,[126,127,128...|[20.7067715360549...|[0.99999999898336...|       0.0|
|  0.0|(692,[126,127,128...|[23.5116910517122...|[0.99999999993848...|       0.0|
|  0.0|(692,[127,128,129...|[20.8660222560481...|[0.99999999913303...|       0.0|
|  0.0|(692,[129,130,131...|[15.8572605148261...|[0.99999987019865...|       0.0|
|  0.0|(692,[152,153,154...|[14.2069485846102...|[0.99999932391650...|       0.0|
|  0.0|(692,[153

In [16]:
 predictions_and_labels.predictions.select("label","prediction").show()

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
+-----+----------+
only showing top 20 rows



In [17]:
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator

In [18]:
evaluator=BinaryClassificationEvaluator(rawPredictionCol="prediction",labelCol="label")
# For multiclass
#evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label',
#                                             metricName='accuracy')

In [19]:
accuracy=evaluator.evaluate(predictions_and_labels.predictions)

In [20]:
print("accuracy : ",accuracy*100," %")

accuracy :  100.0  %
